
# Identifying Spectral Lines in MIRI JWST Data

**Target:** NGC 7469  
**Instrument:** JWST / MIRI / IFU  
**Author:** Satyapriya Das  
**Date:** June 16, 2025

---

This notebook walks through the analysis pipeline for extracting and interpreting mid-infrared spectra from JWST/MIRI data cubes for the active galaxy NGC 7469.

---



## 1. Object Identification and Background

We first gather basic properties of NGC 7469 using NED or Simbad:

| Property | Value |
|---------|--------|
| RA, Dec | 23h 03m 15.6s, +08° 52′ 26″ |
| Distance | ~66 Mpc |
| Redshift (z) | ~0.0163 |
| Category | Seyfert 1 Galaxy (AGN) |

The mid-infrared (MIR) regime is crucial for AGN/starburst studies due to its ability to penetrate dust and trace both thermal and ionized features.


## 2. Load FITS Files and Compute Pixel Scale

In [ ]:

from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

z = 0.0163
filename = "c1006_channel1.fits"

with fits.open(filename) as hdul:
    hdr = hdul['SCI'].header
    cdelt1 = abs(hdr['CDELT1']) * 3600  # arcsec/pixel

scale = cosmo.kpc_proper_per_arcmin(z).to(u.pc/u.arcsec)
pix_scale_pc = cdelt1 * scale

print(f"Pixel scale: {cdelt1:.3f} arcsec/pixel")
print(f"Pixel scale in parsecs: {pix_scale_pc:.2f}")


## 3. Extract Spectra from Region Files

In [ ]:

import numpy as np
import pandas as pd
from astropy.wcs import WCS
from regions import Regions

def extract_spectrum_from_region(data_cube, wcs, region_file):
    region = Regions.read(region_file, format='ds9')[0]
    mask = region.to_mask(mode='center')
    extracted_flux = []

    for lam in range(data_cube.shape[0]):
        flux_slice = data_cube[lam]
        cutout = mask.cutout(flux_slice, fill_value=np.nan)
        value = np.nanmean(cutout)
        extracted_flux.append(value)

    return np.array(extracted_flux)


## 4. Plot Spectra and Overlay Known Emission Lines

In [ ]:

import matplotlib.pyplot as plt

df = pd.read_csv("c1006_channel1_region1.csv")
wavelengths = df['Wavelength [micron]']
flux = df['Flux']

lines = {
    'PAH 6.2': 6.2,
    'PAH 7.7': 7.7,
    'PAH 8.6': 8.6,
    '[S IV] 10.51': 10.51,
    'PAH 11.3': 11.3,
    '[Ne II] 12.81': 12.81,
    '[Ne V] 14.32': 14.32,
    '[Ne III] 15.55': 15.55,
    '[S III] 18.71': 18.71,
    '[Ne V] 24.31': 24.31,
    '[O IV] 25.89': 25.89,
    '[S III] 33.48': 33.48
}

plt.figure(figsize=(10, 6))
plt.plot(wavelengths, flux, label='Region 1', color='black')

for name, wl in lines.items():
    if wavelengths.min() < wl < wavelengths.max():
        plt.axvline(x=wl, color='red', linestyle='--', alpha=0.5)
        plt.text(wl, max(flux)*0.8, name, rotation=90, fontsize=8, color='red', ha='right')

plt.xlabel('Wavelength [μm]')
plt.ylabel('Flux')
plt.title('Region 1 Spectrum with Emission Lines')
plt.grid(True)
plt.tight_layout()
plt.show()


## 5. Compare Spectra Between Regions

In [ ]:

df1 = pd.read_csv("c1006_channel1_region1.csv")
df2 = pd.read_csv("c1006_channel1_region2.csv")

plt.figure(figsize=(10, 6))
plt.plot(df1['Wavelength [micron]'], df1['Flux'], label='Region 1 (Core)', color='blue')
plt.plot(df2['Wavelength [micron]'], df2['Flux'], label='Region 2 (Ring)', color='green')

plt.xlabel("Wavelength [μm]")
plt.ylabel("Flux")
plt.title("Spectral Comparison: Region 1 vs Region 2")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## 6. Emission Line Summary Table

In [ ]:

def detect_flux(df, wavelength, tol=0.1):
    mask = (df['Wavelength [micron]'] > wavelength - tol) & (df['Wavelength [micron]'] < wavelength + tol)
    if mask.any():
        return df['Flux'][mask].max()
    else:
        return np.nan

rows = []
for name, wl in lines.items():
    f1 = detect_flux(df1, wl)
    f2 = detect_flux(df2, wl)
    row = {
        "Line": name,
        "Wavelength (μm)": wl,
        "Region 1 Flux": f"{f1:.2f}" if not np.isnan(f1) else "—",
        "Region 2 Flux": f"{f2:.2f}" if not np.isnan(f2) else "—",
        "Stronger In": "Region 1" if f1 > f2 else ("Region 2" if f2 > f1 else "Equal/—")
    }
    rows.append(row)

line_table = pd.DataFrame(rows)
line_table.to_csv("emission_line_summary.csv", index=False)
line_table
